In [26]:
import pandas as pd

In [35]:
train_raw = pd.read_csv('../train_clean.csv')
train_raw.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0,22.0,1,0,7.2500,1,0,1,0,1
1,1,38.0,1,0,71.2833,0,0,0,0,0
2,1,26.0,0,0,7.9250,0,0,1,0,1
3,1,35.0,1,0,53.1000,0,0,1,0,0
4,0,35.0,0,0,8.0500,1,0,1,0,1


In [36]:
test_raw = pd.read_csv('../test_clean.csv')
# Obtener la columna 'Survived' del DataFrame de prueba
survived_column = test_raw['Survived']

# Eliminar la columna 'Survived' del DataFrame de prueba
test_raw.drop('Survived', axis=1, inplace=True)

# Insertar la columna 'Survived' al principio del DataFrame de prueba
test_raw.insert(0, 'Survived', survived_column)

test_raw.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0,34.5,0,0,7.8292,1,1,0,0,1
1,1,47.0,1,0,7.0000,0,0,1,0,1
2,0,62.0,0,0,9.6875,1,1,0,1,0
3,0,27.0,0,0,8.6625,1,0,1,0,1
4,1,22.0,1,1,12.2875,0,0,1,0,1


In [52]:
from sklearn.preprocessing import MinMaxScaler

# Creamos el objeto MinMaxScaler
scaler = MinMaxScaler()

# Normalizamos las variables 'Age' y 'Fare' en train_raw
train_raw[['Age', 'Fare']] = scaler.fit_transform(train_raw[['Age', 'Fare']])

# Normalizamos las variables 'Age' y 'Fare' en test_raw
test_raw[['Age', 'Fare']] = scaler.transform(test_raw[['Age', 'Fare']])

In [53]:
train_raw.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0,0.271174,1,0,0.014151,1,0,1,0,1
1,1,0.472229,1,0,0.139136,0,0,0,0,0
2,1,0.321438,0,0,0.015469,0,0,1,0,1
3,1,0.434531,1,0,0.103644,0,0,1,0,0
4,0,0.434531,0,0,0.015713,1,0,1,0,1


In [54]:
test_raw.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0,0.428248,0,0,0.015282,1,1,0,0,1
1,1,0.585323,1,0,0.013663,0,0,1,0,1
2,0,0.773813,0,0,0.018909,1,1,0,1,0
3,0,0.334004,0,0,0.016908,1,0,1,0,1
4,1,0.271174,1,1,0.023984,0,0,1,0,1


In [55]:
# Obtener los títulos de las columnas de train_raw y test_raw
columnas_train = train_raw.columns.tolist()
columnas_test = test_raw.columns.tolist()

# Verificar si los títulos de las columnas son iguales
if columnas_train == columnas_test:
    print("Los títulos de las columnas son iguales en train_raw y test_raw.")
else:
    print("Los títulos de las columnas son diferentes en train_raw y test_raw.")

Los títulos de las columnas son iguales en train_raw y test_raw.


In [56]:
# Paso 1: Dividir los datos de entrenamiento en características (X_train) y la variable objetivo (y_train)
X_train = train_raw.drop('Survived', axis=1)
y_train = train_raw['Survived']

### Modelo Random Forest Básico sin busqueda de optimización


In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Paso 2: Dividir los datos de entrenamiento en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Paso 3: Instanciar un clasificador Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Paso 4: Ajustar el clasificador a los datos de entrenamiento
clf.fit(X_train, y_train)

# Paso 5: Evaluar el rendimiento del modelo en los datos de validación
y_pred_val = clf.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)
print("Accuracy del modelo Random Forest en los datos de validación:", accuracy_val)

Accuracy del modelo Random Forest en los datos de validación: 0.8156424581005587


### Búsqueda de mejores parámetros (utilizando Cross Validation)

Los parámetros más relevantes del modelo son:

- n_estimators: El número de árboles en el bosque.
- max_depth: La profundidad máxima de cada árbol.
- max_features: El número máximo de características a considerar al dividir un nodo.
- min_samples_split: El número mínimo de muestras requeridas para dividir un nodo interno.
- min_samples_leaf: El número mínimo de muestras requeridas para estar en un nodo hoja.
- bootstrap: Si se debe o no utilizar el método de bootstrapping para construir árboles.

In [58]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Instanciar el clasificador Random Forest
rfc = RandomForestClassifier(random_state=42)

# Definir los parámetros a probar
param_grid = {
    'n_estimators': [5, 10, 15, 20, 25, 50],
    'max_depth': [None, 2, 3, 5, 7],
    'max_features': [2, 3, 5, 7, 9]
}

# Instanciar la búsqueda de cuadrícula
grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, scoring='accuracy')

# Ajustar la búsqueda de cuadrícula a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor modelo
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)

best_model = grid_search.best_estimator_

Mejores parámetros: {'max_depth': 5, 'max_features': 7, 'n_estimators': 25}


In [61]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Instanciar el clasificador Random Forest con los mejores parámetros
best_params = {'max_depth': 5, 'max_features': 7, 'n_estimators': 25}
best_model = RandomForestClassifier(**best_params, random_state=42)

# Entrenar el modelo con todos los datos de entrenamiento
best_model.fit(X_train, y_train)

# Calcular y mostrar el accuracy del modelo en los datos de entrenamiento (si lo deseas)
train_predicciones = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predicciones)
print("Accuracy del modelo en los datos de entrenamiento:", train_accuracy)


Accuracy del modelo en los datos de entrenamiento: 0.8679775280898876


In [62]:
# Hacer predicciones en los datos de prueba
predicciones_test = best_model.predict(test_raw)

# Imprimir las predicciones
print(predicciones_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Survived
